In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import wandb
import joblib
import os

In [2]:
# Model
import torch.nn as nn
class SEBlock(nn.Module):
    """ Squeeze-and-Excitation Block """
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        scale = self.se(x)
        return x * scale

class MultiHeadAttention(nn.Module):
    """ Multi-Head Attention Module """
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.attention = nn.MultiheadAttention(d_model, num_heads, batch_first=True)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        return attn_output

class RadioNet(nn.Module):
    def __init__(self, num_classes):
        super(RadioNet, self).__init__()

        # Separate Convolutional Pathways for I and Q
        self.q_conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            SEBlock(64),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            SEBlock(256),
            nn.MaxPool2d(2, stride=2)
        )

        self.i_conv = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            SEBlock(64),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            SEBlock(256),
            nn.MaxPool2d(2, stride=2)
        )

        self.feature_size = self._get_conv_output((1, 32, 32))

        # Bidirectional LSTM with Layer Normalization
        self.lstm = nn.LSTM(self.feature_size * 2, 512, num_layers=2, 
                            batch_first=True, bidirectional=True, dropout=0.3)
        self.layer_norm = nn.LayerNorm(1024)  # Layer normalization after LSTM

        # Multi-Head Attention with multiple heads
        self.multi_head_attn = MultiHeadAttention(1024, num_heads=8)

        # Enhanced Fully Connected Layers with Dense Connections
        self.fc = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.LeakyReLU(0.1)
        )

        self.output = nn.Linear(64, num_classes)

    def _get_conv_output(self, shape):
        input = torch.rand(1, *shape)
        output = self.q_conv(input)
        return int(torch.numel(output) / output.shape[0])

    def forward(self, i_input, q_input):
        q = self.q_conv(q_input)
        q = q.view(q.size(0), -1)

        i = self.i_conv(i_input)
        i = i.view(i.size(0), -1)

        combined = torch.cat((q, i), dim=1)
        combined = combined.unsqueeze(1)  # Add sequence dimension

        lstm_out, _ = self.lstm(combined)
        lstm_out = self.layer_norm(lstm_out)

        # Apply Multi-Head Attention
        attn_output = self.multi_head_attn(lstm_out)
        context = torch.sum(attn_output, dim=1)  # Sum up the attended output

        x = self.fc(context)
        x = self.output(x)

        return torch.log_softmax(x, dim=1)

# def create_model(num_classes):
#     model = RadioNet(num_classes)
#     learning_rate = 0.0003
#     optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
#     loss_fn = nn.CrossEntropyLoss()
#     scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
#     return model, optimizer, loss_fn, scheduler

In [3]:
import h5py
# Read the dataset
dataset_file = h5py.File("/kaggle/input/radioml2018/GOLD_XYZ_OSC.0001_1024.hdf5", "r")

# Base modulation classes
base_modulation_classes = [
    'OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK',
    '16APSK', '32APSK', '64APSK', '128APSK', '16QAM', '32QAM', '64QAM',
    '128QAM', '256QAM', 'AM-SSB-WC', 'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC',
    'FM', 'GMSK', 'OQPSK'
]

# Selected modulation classes
selected_modulation_classes = [
    '4ASK', 'BPSK', 'QPSK', '16PSK', '16QAM', 'FM', 'AM-DSB-WC', '32APSK'
]

# Get the indices of selected modulation classes
selected_classes_id = [base_modulation_classes.index(cls) for cls in selected_modulation_classes]

In [4]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Classes
num_classes = len(selected_modulation_classes)

# Load the model
trained_model = RadioNet(num_classes=num_classes)

# Load the state dict, mapping it to the available device
trained_model.load_state_dict(torch.load('/kaggle/input/radionet/pytorch/default/1/model_checkpoint.pth', map_location=device))

# Move the model to the appropriate device
trained_model = trained_model.to(device)

trained_model.eval()

/tmp/ipykernel_17/3816812659.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_model.load_state_dict(torch.load('/kaggle/input/radionet/pytorch/default/1/model_ch

RadioNet(
  (q_conv): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): SEBlock(
      (se): Sequential(
        (0): AdaptiveAvgPool2d(output_size=1)
        (1): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1))
        (2): ReLU()
        (3): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1))
        (4): Sigmoid()
      )
    )
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.1)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.1)

In [5]:
# Freeze the parameters of the pre-trained model
for param in trained_model.parameters():
    param.requires_grad = False

In [6]:
class FrozenFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.model.eval()  # Ensure the model is in evaluation mode
        with torch.no_grad():  # Disable gradient computation
            features_list = []
            for batch in DataLoader(X, batch_size=32):
                batch = batch.to(device)
                i_input = batch[:, :, :, 0].unsqueeze(1)
                q_input = batch[:, :, :, 1].unsqueeze(1)
                
                features = self.model.fc(self.model.multi_head_attn(self.model.layer_norm(self.model.lstm(torch.cat((
                    self.model.q_conv(q_input).view(q_input.size(0), -1),
                    self.model.i_conv(i_input).view(i_input.size(0), -1)
                ), dim=1).unsqueeze(1))[0])).sum(dim=1))
                
                features_list.append(features.cpu().numpy())
            
            return np.vstack(features_list)

In [7]:
# Pipeline
def create_pipeline():
    feature_extractor = FrozenFeatureExtractor(trained_model)
    xgb_classifier = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    
    pipeline = Pipeline([
        ('feature_extraction', feature_extractor),
        ('xgb_classifier', xgb_classifier)
    ])
    
    return pipeline

In [8]:
def train_and_evaluate(X, y, n_splits=5, save_path='trained_pipeline.joblib'):
    wandb.init(project="radioml-xgboost-pipeline", name="xgboost-frozen-feature-pipeline")
    
    pipeline = create_pipeline()
    
    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X, y, cv=n_splits, scoring='accuracy')
    
    print(f"Cross-validation scores: {cv_scores}")
    print(f"Mean CV score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    
    wandb.log({
        "mean_cv_score": cv_scores.mean(),
        "std_cv_score": cv_scores.std(),
        "cv_scores": cv_scores.tolist()
    })
    
    # Train on full dataset
    pipeline.fit(X, y)
    
    # Make predictions
    y_pred = pipeline.predict(X)
    
    # Calculate accuracy
    accuracy = accuracy_score(y, y_pred)
    print(f"Full dataset accuracy: {accuracy:.4f}")
    
    wandb.log({
        "full_dataset_accuracy": accuracy
    })
    
    # Log classification report
    class_report = classification_report(y, y_pred, output_dict=True)
    wandb.log({"classification_report": wandb.Table(dataframe=pd.DataFrame(class_report).transpose())})
    
    # Log feature importances
    feature_imp = pipeline.named_steps['xgb_classifier'].feature_importances_
    wandb.log({"feature_importance": wandb.plot.bar(
        wandb.Table(data=[[f"feature_{i}", imp] for i, imp in enumerate(feature_imp)],
                    columns=["feature", "importance"]),
        "feature",
        "importance",
        title="Feature Importances"
    )})
    
    # Save the trained pipeline
    joblib.dump(pipeline, save_path)
    print(f"Trained pipeline saved to {save_path}")
    
    wandb.finish()
    
    return pipeline

In [9]:
def load_pipeline(load_path='trained_pipeline.joblib'):
    if os.path.exists(load_path):
        pipeline = joblib.load(load_path)
        print(f"Loaded pipeline from {load_path}")
        return pipeline
    else:
        print(f"No saved pipeline found at {load_path}")
        return None

In [10]:
# wandb login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_api_key") 
wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
# from torch.utils.data import Dataset, DataLoader
# class RadioMLDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.from_numpy(X).float().to(device)
#         self.y = torch.from_numpy(y.values).float().to(device)
    
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

In [12]:
# Number of SNRs (from 30 SNR to 22 SNR)
N_SNR = 4 

# Initialize placeholders for data
X_data = None
y_data = None

# Loop through selected modulation classes
for id in selected_classes_id:
    # Load data slices based on indices
    X_slice = dataset_file['X'][(106496*(id+1) - 4096*N_SNR) : 106496*(id+1)]
    y_slice = dataset_file['Y'][(106496*(id+1) - 4096*N_SNR) : 106496*(id+1)]
    
    # Concatenate the slices to build the dataset
    if X_data is not None:
        X_data = np.concatenate([X_data, X_slice], axis=0)
        y_data = np.concatenate([y_data, y_slice], axis=0)
    else:
        X_data = X_slice
        y_data = y_slice

# Reshape the X_data to the required shape (e.g., 32x32 with 2 channels)
X_data = X_data.reshape(len(X_data), 32, 32, 2)

# Convert y_data to a DataFrame for easier manipulation
y_data_df = pd.DataFrame(y_data)

# Drop columns where the sum is 0 (i.e., no modulation class data in that column)
for column in y_data_df.columns:
    if sum(y_data_df[column]) == 0:
        y_data_df = y_data_df.drop(columns=[column])

# Assign the remaining columns to match the selected modulation classes
y_data_df.columns = selected_modulation_classes

y_indices = np.argmax(y_data_df.values, axis=1)

In [13]:
# Main execution
trained_pipeline = train_and_evaluate(X_data, y_indices)

# Example of loading the pipeline
loaded_pipeline = load_pipeline()
if loaded_pipeline:
    # Use the loaded pipeline for predictions
    new_predictions = loaded_pipeline.predict(X_data)
    print("Made predictions using loaded pipeline")

wandb: Currently logged in as: devcode03 (devcode03-gujarat-technological-university). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.18.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240912_180536-zsab8qfm
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run xgboost-frozen-feature-pipeline
wandb: ⭐️ View project at https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost-pipeline
wandb: 🚀 View run at https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost-pipeline/runs/zsab8qfm


Cross-validation scores: [0.99057791 0.99042533 0.99023423 0.99027237 0.98893721]
Mean CV score: 0.9901 (+/- 0.0012)
Full dataset accuracy: 0.9960
Trained pipeline saved to trained_pipeline.joblib


wandb:                                                                                
wandb: 
wandb: Run history:
wandb: full_dataset_accuracy ▁
wandb:         mean_cv_score ▁
wandb:          std_cv_score ▁
wandb: 
wandb: Run summary:
wandb: full_dataset_accuracy 0.99599
wandb:         mean_cv_score 0.99009
wandb:          std_cv_score 0.00059
wandb: 
wandb: 🚀 View run xgboost-frozen-feature-pipeline at: https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost-pipeline/runs/zsab8qfm
wandb: ⭐️ View project at: https://wandb.ai/devcode03-gujarat-technological-university/radioml-xgboost-pipeline
wandb: Synced 5 W&B file(s), 2 media file(s), 2 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240912_180536-zsab8qfm/logs
wandb: WARNING The new W&B backend becomes opt-out in version 0.18.0; try it out with `wandb.require("core")`! See https://wandb.me/wandb-core for more information.
/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: Futur

Loaded pipeline from trained_pipeline.joblib
Made predictions using loaded pipeline
